In [1]:
import pandas as pd
import re

In [2]:
soa_statement_df = pd.read_csv(r"D:\For Auto Recon\SOA REPORT FOR RECONCILE Phase1-20230807T053702Z-001\SOA REPORT FOR RECONCILE Phase1\Commission Report (08_31_2020 to 07_15_2021) Kumari Bank.csv",encoding='latin-1')
# Specify the Excel file path
file_path = r"D:\For Auto Recon\BANK STATEMENT REPORT FOR RECONCILE Phase1-20230807T053815Z-001\BANK STATEMENT REPORT FOR RECONCILE Phase1\kumari bank 31-08-2020 to 15-07-2021.xlsx"
# Read all sheets into a dictionary of DataFrames (sheet_name=None reads all sheets)
all_sheets = pd.read_excel(file_path, sheet_name=None)

# Access each sheet's DataFrame from the dictionary
bank_statement_df = all_sheets['Account Statement Card']

In [3]:
bank_statement_df.head()

,Tran dates,Acct No,Acct Name,Tran Particular,Tran Id,Card Number,Ref Num,Debit Amount,Credit Amount,Tran Remarks
0,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,100000010754,S963852,9841789825,3106533.0,NaN,10.0,08-31 18:48:02//439128
1,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,NEPAL PAYMENT SOLUTION,S963964,shashithp86@gmail.c,234010.0,NaN,10.0,08-31 18:52:14//439128
2,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,NEPAL PAYMENT SOLUTION,S964349,SARASWOTIK,289814.0,NaN,5000.0,08-31 19:26:29//439128
3,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,100000010837,S964806,9847632287,3107042.0,NaN,1000.0,08-31 20:06:06//439128
4,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,100000010847,S965182,9849666654,3107203.0,NaN,2000.0,08-31 20:33:30//439128


In [4]:
bank_statement_df['Tran Particular'].count()

18808

In [5]:
for index, row in bank_statement_df.iterrows():
        if "1000000" in str(row["Tran Particular"]):
            id_matches = re.findall(r'10*[0-9]{5}', str(row["Tran Particular"]))
            if id_matches:
                id = id_matches[0].replace("1000000", "")
                bank_statement_df.loc[index, 'Transaction Id'] = id
        elif "100000" in str(row["Tran Particular"]):
            id_matches = re.findall(r'10*[0-9]{6}', str(row["Tran Particular"]))
            if id_matches:
                id = id_matches[0].replace("100000", "")
                bank_statement_df.loc[index, 'Transaction Id'] = id
        elif "1000000" in str(row["Ref Num"]):
            id_matches = re.findall(r'10*[0-9]{5}', str(row["Ref Num"]))
            if id_matches:
                id = id_matches[0].replace("1000000", "")
                bank_statement_df.loc[index, 'Transaction Id'] = id
        elif "100000" in str(row["Ref Num"]):
            id_matches = re.findall(r'10*[0-9]{6}', str(row["Ref Num"]))
            if id_matches:
                id = id_matches[0].replace("100000", "")
                bank_statement_df.loc[index, 'Transaction Id'] = id

In [6]:
bank_statement_df['Transaction Id'].count()

18684

In [7]:
bank_statement_df.rename(columns={'Tran dates': 'Date'}, inplace=True)

In [8]:
def standard_format(row):
        if row['Debit Amount'] > 0:
            mode = 'DR'
            amount = row['Debit Amount']
        elif row['Credit Amount'] > 0:
            mode = 'CR'
            amount = row['Credit Amount']
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format, axis=1)

In [9]:
bank_statement_df.drop(['Credit Amount','Debit Amount'],inplace=True, axis = 1)

In [12]:
bank_statement_df.columns


Index(['Date', 'Acct No', 'Acct Name', 'Tran Particular', 'Tran Id',
       'Card Number', 'Ref Num', 'Tran Remarks', 'Transaction Id', 'Mode',
       'Amount'],
      dtype='object')

In [17]:
# Convert 'Date' column to datetime if not already done
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'])
bank_statement_df['Converted Date'] = bank_statement_df['Date'].dt.strftime('%d/%m/%y')

In [18]:
bank_statement_df.head()

,Date,Acct No,Acct Name,Tran Particular,Tran Id,Card Number,Ref Num,Tran Remarks,Transaction Id,Mode,Amount,Converted Date
0,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,100000010754,S963852,9841789825,3106533.0,08-31 18:48:02//439128,10754,CR,10.0,31/08/20
1,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,NEPAL PAYMENT SOLUTION,S963964,shashithp86@gmail.c,234010.0,08-31 18:52:14//439128,NaN,CR,10.0,31/08/20
2,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,NEPAL PAYMENT SOLUTION,S964349,SARASWOTIK,289814.0,08-31 19:26:29//439128,NaN,CR,5000.0,31/08/20
3,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,100000010837,S964806,9847632287,3107042.0,08-31 20:06:06//439128,10837,CR,1000.0,31/08/20
4,2020-08-31,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,100000010847,S965182,9849666654,3107203.0,08-31 20:33:30//439128,10847,CR,2000.0,31/08/20


In [ ]:
18808-18684

124

In [ ]:
bank_statement_df.tail()

,Date,Acct No,Acct Name,Tran Particular,Tran Id,Card Number,Ref Num,Tran Remarks,Transaction Id,Mode,Amount
18804,2021-07-15,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,100000730251,S695459,9825246768,7225678.0,07-16 06:41:23//439128,730251,CR,1000.00
18805,2021-07-15,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,100000730249,S695542,9809448802,7225702.0,07-16 06:44:43//439128,730249,CR,1800.00
18806,2021-07-15,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,100000730258,S695587,9809448802,7225722.0,07-16 06:47:14//439128,730258,CR,1800.00
18807,2021-07-15,9.984531e+12,NEPAL PAYMENT SOLUTION SETTLEMENT ACCOUNT,NET NPS LOAD BAISAKHJESTHA,S664841,NaN,NaN,NaN,NaN,DR,15041.53
18808,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DR,89718068.40


In [ ]:
soa_statement_df.head()

,Unnamed: 0,Transaction Id,Opening Balance,Mode,Transaction Type,Amount,Closing Blance,Remarks,Date
0,1,'257535,-237724.49,DR,Gateway,200.0,-237924.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 3:51:00 PM
1,2,'257552,-237924.49,DR,Gateway,1000.0,-238924.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 3:58:00 PM
2,3,'257573,-238924.49,DR,Gateway,490.0,-239414.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 4:08:01 PM
3,4,'257594,-239414.49,DR,Gateway,1000.0,-240414.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 4:16:00 PM
4,5,'257607,-240414.49,DR,Gateway,1000.0,-241414.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 4:24:00 PM


In [ ]:
soa_statement_df['Transaction Id'] =  soa_statement_df['Transaction Id'].str.strip("'").dropna()
soa_statement_df

,Unnamed: 0,Transaction Id,Opening Balance,Mode,Transaction Type,Amount,Closing Blance,Remarks,Date
0,1,257535,-237724.49,DR,Gateway,200.0,-237924.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 3:51:00 PM
1,2,257552,-237924.49,DR,Gateway,1000.0,-238924.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 3:58:00 PM
2,3,257573,-238924.49,DR,Gateway,490.0,-239414.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 4:08:01 PM
3,4,257594,-239414.49,DR,Gateway,1000.0,-240414.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 4:16:00 PM
4,5,257607,-240414.49,DR,Gateway,1000.0,-241414.49,Balance Transferred from NPS Settlement Accoun...,1/19/2021 4:24:00 PM
...,...,...,...,...,...,...,...,...,...
18742,18743,729880,-359522.61,DR,Gateway,500.0,-360022.61,Balance Transferred from NPS Settlement Accoun...,7/15/2021 8:00:00 PM
18743,18744,729888,-360022.61,DR,Gateway,500.0,-360522.61,Balance Transferred from NPS Settlement Accoun...,7/15/2021 8:03:00 PM
18744,18745,730069,-360522.61,DR,Gateway,100.0,-360622.61,Balance Transferred from NPS Settlement Accoun...,7/15/2021 9:42:00 PM
18745,18746,730101,-360622.61,DR,Gateway,100.0,-360722.61,Balance Transferred from NPS Settlement Accoun...,7/15/2021 10:09:00 PM


In [ ]:
bank_statement_df.columns

Index(['Date', 'Acct No', 'Acct Name', 'Tran Particular', 'Tran Id',
       'Card Number', 'Ref Num', 'Tran Remarks', 'Transaction Id', 'Mode',
       'Amount'],
      dtype='object')

In [ ]:
for index1 , bank_statement in bank_statement_df.iterrows():
    if (bank_statement['Transaction Id']) in (soa_statement_df['Transaction Id'].values):
        bank_statement_df.loc[index1,'Matched_TID'] = bank_statement['Transaction Id']
        continue
    bank_statement_df.loc[index1,'Unmatched_TID'] = bank_statement['Transaction Id']
    
  
print(bank_statement_df['Unmatched_TID'].count())  
print(bank_statement_df['Matched_TID'].count())

1833
16851


In [ ]:
for index1, bank_statement_soa in soa_statement_df.iterrows():
    transaction_id = bank_statement_soa['Transaction Id']

    if transaction_id in bank_statement_df['Transaction Id'].values:
        soa_statement_df.loc[index1,'Matched_tid'] = transaction_id
        continue
    soa_statement_df.loc[index1,'Unmatched_tid'] = transaction_id

print(soa_statement_df['Matched_tid'].count())
print(soa_statement_df['Unmatched_tid'].count())

16850
1896
